# Data manipulation with the ORM

In [14]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session
from sqlalchemy import create_engine, MetaData, insert
from sqlalchemy import Table, Column, Integer, String, ForeignKey

In [15]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)
session = Session(engine)

In [16]:
# The Declarative Base refers to a MetaData collection that is created for us automatically
class Base(DeclarativeBase):
    pass

# MetaData collection is accessible via the DeclarativeBase.metadata class-level attribute.
Base.metadata

MetaData()

In [17]:
# Declarative table configuration
class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str | None]
    addresses: Mapped[list["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [18]:
Base.metadata.create_all(engine)

2024-08-13 21:27:46,536 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-13 21:27:46,537 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-08-13 21:27:46,538 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 21:27:46,539 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-08-13 21:27:46,540 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 21:27:46,541 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-08-13 21:27:46,542 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 21:27:46,543 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-08-13 21:27:46,544 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-13 21:27:46,546 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-08-13 21:27:46,547 INFO sqlalchemy.engine.Engine [no key 0.00119s] ()
2024-08-13 21:27:46,548 INFO sqlalchemy.engine.

In [19]:
squidward = User(name="squidward", fullname="Squidward Tentacles")
krabs = User(name="ehkrabs", fullname="Eugene H. Krabs")

squidward, krabs

(User(id=None, name='squidward', fullname='Squidward Tentacles'),
 User(id=None, name='ehkrabs', fullname='Eugene H. Krabs'))

To illustrate the addition process step by step, we will create a Session without using a context manager (and hence we must make sure we close it later!):

In [20]:
# The objects are then added to the Session using the Session.add() method.
# When this is called, the objects are in a state known as pending and have not been inserted yet:
session.add(squidward)
session.add(krabs)

In [21]:
# When we have pending objects, we can see this state by
# looking at a collection on the Session called Session.new:
session.new

IdentitySet([User(id=None, name='squidward', fullname='Squidward Tentacles'), User(id=None, name='ehkrabs', fullname='Eugene H. Krabs')])

In [22]:
# The Session makes use of a pattern known as unit of work.
# This generally means it accumulates changes one at a time, but does not actually communicate them to the database until needed.
# This allows it to make better decisions about how SQL DML should be emitted in the transaction based on a given set of pending changes.
# When it does emit SQL to the database to push out the current set of changes, the process is known as a flush.
session.flush()

2024-08-13 21:27:46,595 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-13 21:27:46,598 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-08-13 21:27:46,599 INFO sqlalchemy.engine.Engine [generated in 0.00015s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('squidward', 'Squidward Tentacles')
2024-08-13 21:27:46,600 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-08-13 21:27:46,601 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('ehkrabs', 'Eugene H. Krabs')


Above we observe the Session was first called upon to emit SQL, so it created a new transaction and emitted the appropriate INSERT statements for the two objects. The transaction now remains open until we call any of the Session.commit(), Session.rollback(), or Session.close() methods of Session.

It is usually **unnecessary** as the Session features a behavior known as autoflush, which we will illustrate later. It also flushes out changes whenever Session.commit() is called.

In [23]:
# Autogenerated primary key attributes
squidward.id, krabs.id

(1, 2)

In [24]:
session.commit()

2024-08-13 21:27:46,627 INFO sqlalchemy.engine.Engine COMMIT


In [25]:
session.close()

In [ ]:
# entities are detached after the session is closed
# squidward.id, krabs.id # DetachedInstanceError